In [1]:
import os
import polars as pl
import pandas as pd
import os.path as path
from spirepy import Study
from spirepy.sample import Sample
from micom.viz import plot_growth
from micom.workflows import grow
import gzip
from Bio import SeqIO
from glob import glob

from micom.qiime_formats import load_qiime_medium
from micom.data import test_medium
from micom.workflows import build, build_database

# Sample definition and manifest generation

In [2]:
study_name = "Lloyd-Price_2019_HMP2IBD"
sample_name = "SAMN07510031"
study = Study(study_name)
data_folder = "/home/alexandre/Documents/projects/micro_fr_pred/data/"

In [3]:
sample = next((x for x in study.samples if x.id == sample_name), None)
out_folder = path.join("/home/alexandre/Documents/projects/micro_fr_pred/data/Lloyd-Price_2019_HMP2IBD",sample.id,"simulations")
sample

Sample id: SAMN07510031 	Study: ['Lloyd-Price_2019_HMP2IBD']

In [4]:
def generate_manifest(sample: Sample):
        manif = []
        reconstruction_folder = path.join(data_folder,study_name,sample.id,"reconstructions")
        mag_folder = path.join(data_folder,study_name,sample.id,"mags")
        abun = get_abundances(sample)
        for genome in sample.mags.iter_rows(named=True):
            manif.append(
                [
                    genome["spire_id"],
                    genome["domain"],
                    genome["phylum"],
                    genome["class"],
                    genome["order"],
                    genome["family"],
                    genome["genus"],
                    genome["species"],
                    path.join(reconstruction_folder, f"{genome['spire_id']}.xml"),
                    genome["sample_id"],
                    abun[path.join(mag_folder, f"{genome['spire_id']}.fa.gz")],
                ]
            )
        manifest = pd.DataFrame(
            manif,
            columns=[
                "id",
                "kingdom",
                "phylum",
                "class",
                "order",
                "family",
                "genus",
                "species",
                "file",
                "sample_id",
                "abundance",
            ],
        )
        manifest.groupby("sample_id")
        manifest["abundance"] = [
            float(i) / sum(manifest["abundance"]) for i in manifest["abundance"]
        ]
        manifest["abundance"] = manifest["abundance"] * 1000
        return manifest

def get_abundances(sample: Sample):
    abundances = {}
    mag_folder = path.join(data_folder, study_name, sample.id, "mags")
    depths = pl.read_csv(
        f"/home/alexandre/microbiome/global_data_spire/SPIRE/studies/{study_name}/psa_megahit/psb_metabat2/{sample.id}_aligned_to_{sample.id}.depths",
        separator="\t",
    )
    for f in sorted(glob(path.join(mag_folder, "*.fa.gz"))):
        with gzip.open(f, "rt") as handle:
            headers = set(rec.id for rec in SeqIO.parse(handle, "fasta"))
        abundance = depths.filter(depths["contigName"].is_in(headers))[
            "totalAvgDepth"
        ].sum()
        abundances[f] = abundance
    return abundances


In [5]:
sample.mags

spire_id,sample_id,spire_cluster,size.spire,size.pg3,size.combined,domain,phylum,class,order,family,genus,species,genome_size,num_contigs,n50,completeness,contamination,gunc_css,gunc_rrs,gene_count
str,str,str,i64,i64,i64,str,str,str,str,str,str,str,i64,i64,i64,f64,f64,f64,f64,i64
"""spire_mag_00374973""","""SAMN07510031""","""specI_v4_02271""",2106,7,2113,"""Bacteria""","""Proteobacteria""","""Gammaproteobacteria""","""Burkholderiales""","""Burkholderiaceae""","""Sutterella""","""Sutterella wadsworthensis_A""",2144463,118,26489,92.76,0.37,0.0,0.92,1954
"""spire_mag_00374977""","""SAMN07510031""","""specI_v4_00250""",5645,72,5717,"""Bacteria""","""Bacteroidota""","""Bacteroidia""","""Bacteroidales""","""Tannerellaceae""","""Parabacteroides""","""Parabacteroides distasonis""",4001358,177,37312,86.18,0.24,0.3,0.91,3363
"""spire_mag_00374978""","""SAMN07510031""","""spire_v1_095_000002204""",2514,0,2514,"""Bacteria""","""Bacteroidota""","""Bacteroidia""","""Bacteroidales""","""Bacteroidaceae""","""Bacteroides""","""Bacteroides ovatus""",6499233,111,116280,93.99,2.79,0.41,0.87,5090
"""spire_mag_00374979""","""SAMN07510031""","""specI_v4_00037""",7711,917,8628,"""Bacteria""","""Actinobacteriota""","""Actinomycetia""","""Actinomycetales""","""Bifidobacteriaceae""","""Bifidobacterium""","""Bifidobacterium longum""",2111246,126,25068,95.37,0.9,0.0,0.91,1856
"""spire_mag_00374980""","""SAMN07510031""","""specI_v4_02498""",1577,7,1584,"""Bacteria""","""Firmicutes_A""","""Clostridia""","""Oscillospirales""","""Ruminococcaceae""","""Faecalibacterium""","""Faecalibacterium prausnitzii_I""",1272441,18,107325,52.4,0.33,0.07,0.8,1105
"""spire_mag_00374982""","""SAMN07510031""","""spire_v1_095_000002205""",426,0,426,"""Bacteria""","""Bacteroidota""","""Bacteroidia""","""Bacteroidales""","""Bacteroidaceae""","""Bacteroides""","""Bacteroides faecis""",4803560,456,13969,80.29,1.7,0.0,0.84,4107
"""spire_mag_00374983""","""SAMN07510031""","""specI_v4_00088""",16781,240,17021,"""Bacteria""","""Bacteroidota""","""Bacteroidia""","""Bacteroidales""","""Bacteroidaceae""","""Bacteroides""","""Bacteroides uniformis""",3535060,32,223998,91.23,0.03,0.0,0.9,2890
"""spire_mag_00374985""","""SAMN07510031""","""specI_v4_00546""",6976,30,7006,"""Bacteria""","""Firmicutes_A""","""Clostridia""","""Lachnospirales""","""Lachnospiraceae""","""Agathobacter""","""Agathobacter faecis""",2050266,95,38081,81.05,0.66,0.0,0.93,2028
"""spire_mag_00374987""","""SAMN07510031""","""specI_v4_00263""",5621,63,5684,"""Bacteria""","""Bacteroidota""","""Bacteroidia""","""Bacteroidales""","""Bacteroidaceae""","""Bacteroides""","""Bacteroides stercoris""",2877764,46,89665,72.94,0.86,0.0,0.93,2364


In [6]:
tax = generate_manifest(sample)
tax

,id,kingdom,phylum,class,order,family,genus,species,file,sample_id,abundance
0,spire_mag_00374973,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Burkholderiaceae,Sutterella,Sutterella wadsworthensis_A,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,76.304352
1,spire_mag_00374977,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Tannerellaceae,Parabacteroides,Parabacteroides distasonis,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,88.821351
2,spire_mag_00374978,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides ovatus,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,185.546435
3,spire_mag_00374979,Bacteria,Actinobacteriota,Actinomycetia,Actinomycetales,Bifidobacteriaceae,Bifidobacterium,Bifidobacterium longum,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,73.587600
4,spire_mag_00374980,Bacteria,Firmicutes_A,Clostridia,Oscillospirales,Ruminococcaceae,Faecalibacterium,Faecalibacterium prausnitzii_I,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,34.724592
5,spire_mag_00374982,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides faecis,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,226.287189
6,spire_mag_00374983,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides uniformis,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,55.573955
7,spire_mag_00374985,Bacteria,Firmicutes_A,Clostridia,Lachnospirales,Lachnospiraceae,Agathobacter,Agathobacter faecis,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,49.972899
8,spire_mag_00374987,Bacteria,Bacteroidota,Bacteroidia,Bacteroidales,Bacteroidaceae,Bacteroides,Bacteroides stercoris,/home/alexandre/Documents/projects/micro_fr_pr...,SAMN07510031,209.181626


In [7]:
manifest = build(
    taxonomy=tax,
    out_folder=f"{out_folder}",
    model_db=None,
    cutoff=0.0001,
    threads=2,
)

manifest

[05/16/25 12:50:18] WARNING  Found existing models for 1 samples. Will skip those. Delete the output    ]8;id=403324;file:///home/alexandre/miniconda3/envs/mtgw/lib/python3.9/site-packages/micom/workflows/build.py\build.py]8;;\:]8;id=897604;file:///home/alexandre/miniconda3/envs/mtgw/lib/python3.9/site-packages/micom/workflows/build.py#98\98]8;;\
                             folder if you would like me to rebuild them.                                          

Output()

Set parameter Username
Set parameter LicenseID to value 2582980
Academic license - for non-commercial use only - expires 2025-11-13
Read LP format model from file /tmp/tmp6lurmqwm.lp
Reading time = 0.02 seconds
: 5491 rows, 13719 columns, 55711 nonzeros


,sample_id,file
0,SAMN07510031,SAMN07510031.pickle


# Western Diet medium

In [8]:
medium = load_qiime_medium("../data/western_diet_gut.qza")
medium

,flux,dilution,metabolite,reaction
reaction,,,,
EX_fru_m,0.014899,0.100,fru_m,EX_fru_m
EX_glc_m,0.014899,0.100,glc_m,EX_glc_m
EX_gal_m,0.014899,0.100,gal_m,EX_gal_m
EX_man_m,0.014899,0.100,man_m,EX_man_m
EX_mnl_m,0.014899,0.100,mnl_m,EX_mnl_m
...,...,...,...,...
EX_glu_D_m,0.100000,0.100,glu_D_m,EX_glu_D_m
EX_gthrd_m,0.100000,0.100,gthrd_m,EX_gthrd_m
EX_h2_m,0.100000,0.100,h2_m,EX_h2_m


In [9]:
res = grow(
    manifest,
    model_folder=out_folder,
    medium=medium,
    tradeoff=1.0,
    threads=2,
)

Output()

Set parameter Username
Set parameter LicenseID to value 2582980
Academic license - for non-commercial use only - expires 2025-11-13
Read LP format model from file /tmp/tmpl9d_b7ru.lp
Reading time = 0.03 seconds
: 5491 rows, 13719 columns, 55711 nonzeros


In [10]:
pl = plot_growth(res, filename=path.join(out_folder,"growth_rates.html"))
pl.view()

In [11]:
from micom.interaction import interactions

ints = interactions(res, taxa='spire_mag_00374987')
ints.sort_values(by="flux", ascending=False).head()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
0,fe3_e,spire_mag_00374987,spire_mag_00374973,co-consumed,2.318718,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
27,pyovd_kt_e,spire_mag_00374987,spire_mag_00374973,co-consumed,2.318718,SAMN07510031,Pyoverdine P putida specific,752.72802,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_pyovd_kt_e
54,fe3pyovd_kt_e,spire_mag_00374987,spire_mag_00374978,provided,2.318718,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e
53,fe3pyovd_kt_e,spire_mag_00374987,spire_mag_00374979,provided,2.318718,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e
52,fe3pyovd_kt_e,spire_mag_00374987,spire_mag_00374980,provided,2.318718,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e


In [12]:
ints = interactions(res, taxa='spire_mag_00374978')
ints.sort_values(by="flux", ascending=False).head()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
5,fe3_e,spire_mag_00374978,spire_mag_00374977,provided,8.882135,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
35,pyovd_kt_e,spire_mag_00374978,spire_mag_00374977,provided,8.882066,SAMN07510031,Pyoverdine P putida specific,752.72802,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_pyovd_kt_e
11,fe3pyovd_kt_e,spire_mag_00374978,spire_mag_00374977,received,8.882066,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e
0,fe3_e,spire_mag_00374978,spire_mag_00374973,provided,7.630435,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
6,fe3pyovd_kt_e,spire_mag_00374978,spire_mag_00374973,received,7.630397,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e


In [13]:
full = interactions(res, taxa=None, threads=8)

full

Output()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
0,acgam1p_e,spire_mag_00374982,spire_mag_00374973,provided,0.000978,SAMN07510031,N-Acetyl-D-glucosamine 1-phosphate,299.171821,8,1,...,NaN,MNXM91871,FZLJPEPAYPUMMR-RTRLPJTCSA-L,cpd02611,R-ALL-532204,HMDB01367,NaN,NaN,NaN,EX_acgam1p_e
1,acgam1p_e,spire_mag_00374982,spire_mag_00374985,provided,0.000103,SAMN07510031,N-Acetyl-D-glucosamine 1-phosphate,299.171821,8,1,...,NaN,MNXM91871,FZLJPEPAYPUMMR-RTRLPJTCSA-L,cpd02611,R-ALL-532204,HMDB01367,NaN,NaN,NaN,EX_acgam1p_e
2,arg__L_e,spire_mag_00374982,spire_mag_00374973,co-consumed,0.001429,SAMN07510031,L-Arginine,175.208900,6,4,...,META:ARG,MNXM70,ODKSFYDXXFIFQN-BYPYZUCNSA-O,"['cpd00051', 'cpd19021']","['R-ALL-113530', 'R-ALL-29468', 'R-ALL-351977'...","['HMDB00517', 'HMDB62762']",NaN,D02982,NaN,EX_arg__L_e
3,arg__L_e,spire_mag_00374982,spire_mag_00374985,co-consumed,0.000152,SAMN07510031,L-Arginine,175.208900,6,4,...,META:ARG,MNXM70,ODKSFYDXXFIFQN-BYPYZUCNSA-O,"['cpd00051', 'cpd19021']","['R-ALL-113530', 'R-ALL-29468', 'R-ALL-351977'...","['HMDB00517', 'HMDB62762']",NaN,D02982,NaN,EX_arg__L_e
4,arg__L_e,spire_mag_00374982,spire_mag_00374977,received,0.001429,SAMN07510031,L-Arginine,175.208900,6,4,...,META:ARG,MNXM70,ODKSFYDXXFIFQN-BYPYZUCNSA-O,"['cpd00051', 'cpd19021']","['R-ALL-113530', 'R-ALL-29468', 'R-ALL-351977'...","['HMDB00517', 'HMDB62762']",NaN,D02982,NaN,EX_arg__L_e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,argp_e,spire_mag_00374987,spire_mag_00374979,provided,0.000677,SAMN07510031,L Arginine phosphate C6H14N4O5P,253.172921,6,4,...,META:L-ARGININE-P,MNXM3663,CCTIOCVIZPCTGO-BYPYZUCNSA-M,cpd03535,NaN,NaN,NaN,NaN,NaN,EX_argp_e
62,co2_e,spire_mag_00374987,spire_mag_00374977,provided,0.029785,SAMN07510031,CO2 CO2,44.009500,1,0,...,META:CARBON-DIOXIDE,MNXM13,CURLTUGMZLYLDI-UHFFFAOYSA-N,cpd00011,"['R-ALL-1132345', 'R-ALL-113528', 'R-ALL-12370...",HMDB01967,NaN,D00004,NaN,EX_co2_e
63,h2s_e,spire_mag_00374987,spire_mag_00374977,provided,0.001365,SAMN07510031,Hydrogen sulfide,34.080880,0,0,...,"['META:CPD-7046', 'META:CPD-846', 'META:HS']",MNXM89582,RWSOTUBLDIXVET-UHFFFAOYSA-M,"['cpd00239', 'cpd24697']","['R-ALL-1614532', 'R-ALL-1655881', 'R-ALL-1663...","['HMDB00598', 'HMDB03276']",NaN,NaN,NaN,EX_h2s_e
64,nh4_e,spire_mag_00374987,spire_mag_00374979,provided,0.009970,SAMN07510031,Ammonium,18.038460,0,1,...,"['META:AMMONIA', 'META:AMMONIUM']",MNXM15,QGZKDVFQNNGYKY-UHFFFAOYSA-O,"['cpd00013', 'cpd19013']","['R-ALL-1132163', 'R-ALL-113561', 'R-ALL-14091...","['HMDB00051', 'HMDB41827']",NaN,"['D02915', 'D02916']",NaN,EX_nh4_e


In [14]:
from micom.interaction import summarize_interactions

summary = summarize_interactions(full)
summary.sort_values(by="n_ints", ascending=False)

,sample_id,focal,partner,class,flux,mass_flux,C_flux,N_flux,n_ints
114,SAMN07510031,spire_mag_00374982,spire_mag_00374977,co-consumed,0.069423,0.014715,0.471615,0.032686,17
34,SAMN07510031,spire_mag_00374977,spire_mag_00374982,co-consumed,0.069423,0.014715,0.471615,0.032686,17
0,SAMN07510031,spire_mag_00374973,spire_mag_00374977,co-consumed,15.275649,6.171320,206.069368,76.328282,15
22,SAMN07510031,spire_mag_00374977,spire_mag_00374973,co-consumed,15.275649,6.171320,206.069368,76.328282,15
112,SAMN07510031,spire_mag_00374982,spire_mag_00374973,co-consumed,0.011848,0.002798,0.105288,0.013413,15
...,...,...,...,...,...,...,...,...,...
150,SAMN07510031,spire_mag_00374983,spire_mag_00374987,received,0.001048,0.000271,0.006290,0.000000,1
48,SAMN07510031,spire_mag_00374978,spire_mag_00374977,co-consumed,0.012187,0.000561,0.000000,0.012187,1
45,SAMN07510031,spire_mag_00374978,spire_mag_00374973,co-consumed,0.048280,0.000049,0.000000,0.000000,1
35,SAMN07510031,spire_mag_00374977,spire_mag_00374982,provided,0.001429,0.000250,0.008572,0.005715,1


In [15]:
summary.groupby("class")["n_ints"].sum()

class
co-consumed    376
provided       184
received       184
Name: n_ints, dtype: int64

In [16]:
summary.groupby("class")["flux"].sum()

class
co-consumed    203.914764
provided       239.073595
received       239.073595
Name: flux, dtype: float64

In [17]:
from micom.viz import plot_focal_interactions

pl = plot_focal_interactions(res, taxon="spire_mag_00374978", filename=path.join(out_folder, "niche.html"))
pl.view()

In [18]:
from micom.viz import plot_mes

pl = plot_mes(res, filename=path.join(out_folder, "mes.html"))
pl.view()

# Fixed medium

In [19]:
from micom.workflows import complete_community_medium

fixed_medium = complete_community_medium(manifest, model_folder=out_folder, medium=medium,
                    community_growth=0.1, min_growth=0.01,
                    max_import=10, threads=2)
fixed_medium

Output()

Set parameter Username
Set parameter LicenseID to value 2582980
Academic license - for non-commercial use only - expires 2025-11-13
Read LP format model from file /tmp/tmpvii_jq92.lp
Reading time = 0.03 seconds
: 5491 rows, 13719 columns, 55711 nonzeros


,reaction,metabolite,description,flux
0,EX_2obut_m,2obut_m,2-Oxobutanoate,0.100000
1,EX_4abz_m,4abz_m,4-Aminobenzoate,0.100000
2,EX_ac_m,ac_m,Acetate,0.100000
3,EX_acmana_m,acmana_m,N-Acetyl-D-mannosamine,0.100000
4,EX_ade_m,ade_m,Adenine,0.100000
...,...,...,...,...
87,EX_ump_m,ump_m,UMP C9H11N2O9P,0.000909
88,EX_ura_m,ura_m,Uracil,0.100000
89,EX_uri_m,uri_m,Uridine,0.100000
90,EX_xan_m,xan_m,Xanthine,0.100000


In [20]:
res_fixed = grow(
    manifest,
    model_folder=out_folder,
    medium=fixed_medium,
    tradeoff=1.0,
    threads=2,
)

Output()

Set parameter Username
Set parameter LicenseID to value 2582980
Academic license - for non-commercial use only - expires 2025-11-13
Read LP format model from file /tmp/tmpdbzjzfau.lp
Reading time = 0.03 seconds
: 5491 rows, 13719 columns, 55711 nonzeros


In [21]:
pl = plot_growth(res_fixed, filename=path.join(out_folder, "growth_rates_fixed_medium.html"))
pl.view()

In [22]:
from micom.interaction import interactions

ints = interactions(res_fixed, taxa='spire_mag_00374987')
ints.sort_values(by="flux", ascending=False).head()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
49,fe3_e,spire_mag_00374987,spire_mag_00374977,provided,8.882135,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
55,fe3pyovd_kt_e,spire_mag_00374987,spire_mag_00374977,received,8.882034,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e
124,pyovd_kt_e,spire_mag_00374987,spire_mag_00374977,provided,8.882034,SAMN07510031,Pyoverdine P putida specific,752.72802,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_pyovd_kt_e
48,fe3_e,spire_mag_00374987,spire_mag_00374973,provided,7.630435,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
123,pyovd_kt_e,spire_mag_00374987,spire_mag_00374973,provided,7.630384,SAMN07510031,Pyoverdine P putida specific,752.72802,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_pyovd_kt_e


In [23]:
ints = interactions(res_fixed, taxa='spire_mag_00374978')
ints.sort_values(by="flux", ascending=False).head()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
115,pyovd_kt_e,spire_mag_00374978,spire_mag_00374979,provided,5.864555,SAMN07510031,Pyoverdine P putida specific,752.72802,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_pyovd_kt_e
51,fe3pyovd_kt_e,spire_mag_00374978,spire_mag_00374987,co-consumed,5.864555,SAMN07510031,Ferrypyoverdine P putida KT2440 specific,808.57302,27,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,EX_fe3pyovd_kt_e
40,fe3_e,spire_mag_00374978,spire_mag_00374977,provided,5.864555,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
39,fe3_e,spire_mag_00374978,spire_mag_00374973,provided,5.864555,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e
43,fe3_e,spire_mag_00374978,spire_mag_00374979,provided,5.864555,SAMN07510031,Iron (Fe3+),55.84500,0,0,...,META:FE+3,MNXM196,VTLYFUHAOXGGBS-UHFFFAOYSA-N,cpd10516,"['R-ALL-111736', 'R-ALL-912516', 'R-ALL-912519...",HMDB12943,NaN,NaN,NaN,EX_fe3_e


In [24]:
full_fixed = interactions(res_fixed, taxa=None, threads=8)

full_fixed

Output()

,metabolite,focal,partner,class,flux,sample_id,name,molecular_weight,C_number,N_number,...,biocyc,metanetx.chemical,inchikey,seed.compound,reactome,hmdb,lipidmaps,kegg.drug,kegg.glycan,reaction
0,acgam1p_e,spire_mag_00374982,spire_mag_00374973,provided,0.001314,SAMN07510031,N-Acetyl-D-glucosamine 1-phosphate,299.171821,8,1,...,NaN,MNXM91871,FZLJPEPAYPUMMR-RTRLPJTCSA-L,cpd02611,R-ALL-532204,HMDB01367,NaN,NaN,NaN,EX_acgam1p_e
1,acgam1p_e,spire_mag_00374982,spire_mag_00374980,provided,0.000357,SAMN07510031,N-Acetyl-D-glucosamine 1-phosphate,299.171821,8,1,...,NaN,MNXM91871,FZLJPEPAYPUMMR-RTRLPJTCSA-L,cpd02611,R-ALL-532204,HMDB01367,NaN,NaN,NaN,EX_acgam1p_e
2,acgam1p_e,spire_mag_00374982,spire_mag_00374985,provided,0.005316,SAMN07510031,N-Acetyl-D-glucosamine 1-phosphate,299.171821,8,1,...,NaN,MNXM91871,FZLJPEPAYPUMMR-RTRLPJTCSA-L,cpd02611,R-ALL-532204,HMDB01367,NaN,NaN,NaN,EX_acgam1p_e
3,arg__L_e,spire_mag_00374982,spire_mag_00374973,co-consumed,0.000902,SAMN07510031,L-Arginine,175.208900,6,4,...,META:ARG,MNXM70,ODKSFYDXXFIFQN-BYPYZUCNSA-O,"['cpd00051', 'cpd19021']","['R-ALL-113530', 'R-ALL-29468', 'R-ALL-351977'...","['HMDB00517', 'HMDB62762']",NaN,D02982,NaN,EX_arg__L_e
4,arg__L_e,spire_mag_00374982,spire_mag_00374977,received,0.000902,SAMN07510031,L-Arginine,175.208900,6,4,...,META:ARG,MNXM70,ODKSFYDXXFIFQN-BYPYZUCNSA-O,"['cpd00051', 'cpd19021']","['R-ALL-113530', 'R-ALL-29468', 'R-ALL-351977'...","['HMDB00517', 'HMDB62762']",NaN,D02982,NaN,EX_arg__L_e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
180,amp_e,spire_mag_00374987,spire_mag_00374985,co-consumed,0.002470,SAMN07510031,AMP C10H12N5O7P,345.205341,10,5,...,"['META:AMP', 'META:AMP-GROUP']",MNXM14,UDMBCSSLTHHNCD-KQYNXXCUSA-L,"['cpd00018', 'cpd22272']","['R-ALL-109275', 'R-ALL-159448', 'R-ALL-164121...",HMDB00045,NaN,D02769,NaN,EX_amp_e
181,cellb_e,spire_mag_00374987,spire_mag_00374982,provided,0.027327,SAMN07510031,Cellobiose,342.296480,12,0,...,META:CELLOBIOSE,MNXM1142,GUBGYTABKSRVRQ-QRZGKKJRSA-N,cpd03845,NaN,HMDB00055,NaN,NaN,G00289,EX_cellb_e
182,cellb_e,spire_mag_00374987,spire_mag_00374979,provided,0.037247,SAMN07510031,Cellobiose,342.296480,12,0,...,META:CELLOBIOSE,MNXM1142,GUBGYTABKSRVRQ-QRZGKKJRSA-N,cpd03845,NaN,HMDB00055,NaN,NaN,G00289,EX_cellb_e
183,cellb_e,spire_mag_00374987,spire_mag_00374985,provided,0.037247,SAMN07510031,Cellobiose,342.296480,12,0,...,META:CELLOBIOSE,MNXM1142,GUBGYTABKSRVRQ-QRZGKKJRSA-N,cpd03845,NaN,HMDB00055,NaN,NaN,G00289,EX_cellb_e


In [25]:
from micom.interaction import summarize_interactions

summary_fixed = summarize_interactions(full_fixed)
summary_fixed.sort_values(by="n_ints", ascending=False)

,sample_id,focal,partner,class,flux,mass_flux,C_flux,N_flux,n_ints
82,SAMN07510031,spire_mag_00374979,spire_mag_00374985,co-consumed,10.528287,4.121802,137.466577,50.143021,24
151,SAMN07510031,spire_mag_00374985,spire_mag_00374979,co-consumed,10.528287,4.121802,137.466577,50.143021,24
71,SAMN07510031,spire_mag_00374979,spire_mag_00374977,co-consumed,14.742111,5.957085,198.906509,73.659365,21
29,SAMN07510031,spire_mag_00374977,spire_mag_00374979,co-consumed,14.742111,5.957085,198.906509,73.659365,21
153,SAMN07510031,spire_mag_00374985,spire_mag_00374980,co-consumed,7.013224,2.812925,93.888244,34.730522,21
...,...,...,...,...,...,...,...,...,...
155,SAMN07510031,spire_mag_00374985,spire_mag_00374980,received,0.020326,0.000020,0.000000,0.000000,1
66,SAMN07510031,spire_mag_00374978,spire_mag_00374987,provided,0.029358,0.001292,0.029358,0.000000,1
157,SAMN07510031,spire_mag_00374985,spire_mag_00374982,provided,0.004117,0.000132,0.000000,0.000000,1
159,SAMN07510031,spire_mag_00374985,spire_mag_00374983,provided,4.997082,4.040506,134.921225,49.970824,1


In [26]:
summary_fixed.groupby("class")["n_ints"].sum()

class
co-consumed    990
provided       198
received       198
Name: n_ints, dtype: int64

In [27]:
summary_fixed.groupby("class")["flux"].sum()

class
co-consumed    242.393062
provided       258.851743
received       258.851743
Name: flux, dtype: float64

In [28]:
from micom.viz import plot_focal_interactions

pl = plot_focal_interactions(res_fixed, taxon="spire_mag_00374978", filename=path.join(out_folder, "niche_fixed_medium.html"))
pl.view()

In [29]:
from micom.viz import plot_mes

pl = plot_mes(res_fixed, filename=path.join(out_folder, "mes_fixed.html"))
pl.view()